# FunOS coverage analysis

This notebook is used to analyze the coverage of FunOS.

In [ ]:
import pandas as pd
import numpy as np
import requests

import json

## Load coverage json file

In [ ]:

working_dir = "."
input_url = "http://jenkins-sw-master.fungible.local/ci/job/scheduled/job/coverage/102/Coverage_20Reports/nucleus/index.functions.html"
# input_url = "http://jenkins-sw-master.fungible.local/ci/job/scheduled/job/coverage/102/Coverage_20Reports/malloc/index.functions.html"

# input_url = "http://jenkins-sw-master.fungible.local/ci/job/scheduled/job/coverage/101/execution/node/25/ws/coverage/nucleus/index.functions.html/*view*/"
# input_url = "http://jenkins-sw-master.fungible.local/ci/job/scheduled/job/coverage/101/execution/node/25/ws/coverage/nucleus/coverage.json"
# input_url = "http://jenkins-sw-master.fungible.local/ci/job/scheduled/job/coverage/89/execution/node/21/ws/coverage/nucleus/coverage.json"
#input_url = "http://jenkins-sw-master.fungible.local/ci/job/scheduled/job/coverage/88/execution/node/22/ws/coverage/nucleus/coverage.json/*view*/"
#input_url = "http://jenkins-sw-master.fungible.local/ci/job/scheduled/job/coverage/80/execution/node/21/ws/coverage/nucleus/coverage.json/*view*/"
# input_url = "http://jenkins-sw-master.fungible.local/ci/job/scheduled/job/coverage/69/execution/node/22/ws/coverage/nucleus/coverage.json/*view*/"


In [ ]:
from requests.auth import HTTPBasicAuth
ID = os.environ["JENKINS_ID"]
PASSWD = os.environ["JENKINS_PASSWD"]
basic = HTTPBasicAuth(ID, PASSWD)
response = requests.get(input_url, auth=basic)
lines = response.text

In [ ]:
lines

In [ ]:
ID

In [ ]:
# df = pd.read_json(lines)

In [ ]:
cov_dict = json.loads(lines)
cov_dict.keys()

# cov_dict['files']

In [ ]:
# cov_dict['files']

In [ ]:
# data frame from list of dict
df = pd.DataFrame(cov_dict['files'])

# set filename as index
df.set_index('filename', inplace=True)

df_org = df.copy()

# drop columns
# df.drop(['branch_covered', 'branch_total', 'function_covered', 'line_covered', 'line_total'], axis=1, inplace=True)

In [ ]:
# df_org['line_covered'].sum()/ df_org['line_total'].sum()

In [ ]:
# sort df by function_percent
# add new column of function_uncovered
df['function_uncovered'] = df['function_total'] - df['function_covered']

df.sort_values(by=['function_percent', 'line_percent', 'branch_percent'], ascending=True, inplace=True)
# show only columns of function_percent

df_tmp = df[['function_percent', 'function_total', 'function_uncovered']]
display(df_tmp.head(50))

In [ ]:
# sum of all function_total column
function_total_sum = df['function_total'].sum()
function_covered_sum = df['function_covered'].sum()
function_percent_computed = function_covered_sum / function_total_sum

print(f' function_total_sum: {function_total_sum}')
print(f' function_covered_sum: {function_covered_sum}')
print(f' function percentage computed: {function_percent_computed}')

In [ ]:
# compute desired function covered and percentage

desired_covered_75 = function_total_sum * 0.75
desired_covered_80 = function_total_sum * 0.8
desired_covered_90 = function_total_sum * 0.9

delta_btwn_now_75 = desired_covered_75 - function_covered_sum
delta_btwn_now_80 = desired_covered_80 - function_covered_sum

print(f' desired_covered_75: {desired_covered_75}')
print(f' desired_covered_80: {desired_covered_80}')
print(f' desired_covered_90: {desired_covered_90}')
print('')
print(f' delta_btwn_now_75: {delta_btwn_now_75}')
print(f' delta_btwn_now_80: {delta_btwn_now_80}')

In [ ]:
# display index of df nucleus/trace_record.c

# display(df.loc['utils/wu/condvar.c'])

In [ ]:
# average of function_percent
# this is not actual summary, since what's been compuated is based on the total functions and covered funtions, not the average of the percentage of each file
# df['function_percent'].mean()

In [ ]:
df_t = df[:50]

In [ ]:
# plot dataframe reverse order
df_t[::-1].plot(kind='barh', figsize=(10, 20), title='Coverage by file')
# df[:30].plot(kind='barh', figsize=(10, 20), title='Function Coverage')

In [ ]:

df.plot(kind='barh', figsize=(10, 20), title='Function Coverage')

## Test parsing html

To extract per function coverage, we need to parse the html file.

In [ ]:
html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

In [ ]:
html_file = "/Users/insop/tmp/coverage_mac_11_28/nucleus/index.adi.c.81c7dd8fb7388abe4acd04e2508ff6ee.html"

In [ ]:
from bs4 import BeautifulSoup

# with open("index.html") as fp:
#     soup = BeautifulSoup(fp, 'html.parser')

In [ ]:
# read text file
with open(html_file, "r", encoding="utf-8") as f:
    # html_doc = f.read()
    soup = BeautifulSoup(f, 'html.parser')


# read text file


In [ ]:
type(lines)

In [ ]:
soup = BeautifulSoup(lines, "html.parser")

In [ ]:
# from bs4 import BeautifulSoup
# soup = BeautifulSoup(html_doc, 'html.parser')

# print(soup.prettify())

In [ ]:
print(soup.a)

In [ ]:
# print(soup.get_text())

In [ ]:
print(soup.find(id="nav"))

In [ ]:
# soup = BeautifulSoup('<b class="boldest">Extremely bold</b>', 'html.parser')
tab = soup.find("table",{"class":"listOfFunctions"})
print(tab)


In [ ]:
# convert bs4 to string

type(str(tab))

In [ ]:
import pandas as pd

dfs = pd.read_html(str(tab))
display(type(dfs))

In [ ]:
dfs[0]

In [ ]:
dfs[0].describe()

In [ ]:
# glob all html files
#
# init main_df
#
# loop:
# . read html
# . extract tab
# . convert to df
# . add df to the main_df

# save main_df to csv